In [2]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
from langgraph.graph import StateGraph, END
from typing import Annotated,TypedDict
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage
from langchain_community.tools.tavily_search import TavilySearchResults

In [5]:
tool = TavilySearchResults(tavily_api_key=os.environ["TAVILY_API_KEY"],max_results=2)
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


/tmp/ipykernel_5032/65549071.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(tavily_api_key=os.environ["TAVILY_API_KEY"],max_results=2)


In [6]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage],operator.add]

In [8]:
class Agent:
    def __init__(self,model,tools,system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm",...)
        graph.add_node("action",...)
        graph.add_conditional_edges(
            "llm",
            ...,
            {True:"action",False:END}
        )
        graph.add_edge("llm","action")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name:t for t in tools}
        self.model = model.bind_tools(tools)
    
    def call_groq(self,state:AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages":[message]}
